In [5]:
import random
import re
from os import sep

import numpy as np
import pandas as pd
import requests
import tmdbsimple as tmdb
from scipy.stats import pearsonr
from surprise import SVD, Dataset, KNNBasic, Reader
from surprise.model_selection import cross_validate
from collections import defaultdict, Counter
from itertools import islice
import collections

In [2]:
tmdb.REQUESTS_SESSION = requests.Session()
tmdb.API_KEY = '1e11e7d4c5f3aad6e459fc0f63bfb0f5'
tmdb.REQUESTS_TIMEOUT = 5

In [3]:
users_df = pd.read_csv(
    "../data/users.txt", names=["user_id", "age", "gender", "occupation"], sep="\t"
)

In [4]:
users_df

,user_id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other
...,...,...,...,...
938,939,26,F,student
939,940,32,M,administrator
940,941,20,M,student
941,942,48,F,librarian


In [107]:
def get_user_type(gender, age, occupation):
    res = 0
    group1 = ['doctor', 'healthcare', 'entertainment']
    group2 = ['engineer', 'programmer', 'scientist', 'technician']
    group3 = ['artist', 'writer', 'librarian', 'homemaker']
    group4 = ['none', 'other', 'student', 'retired']
    group5 = ['executive', 'lawyer', 'administrator', 'salesman', 'marketing']

    if occupation in group1:
        if age < 23:
            res = 1 if gender == 'M' else 2
        elif 24 < age < 40:
            res = 3 if gender == 'M' else 4
        elif 41 < age < 60:
            res = 5 if gender == 'M' else 6
        else:
            res = 7 if gender == 'M' else 8
    elif occupation in group2:
        if age < 23:
            res = 9 if gender == 'M' else 9
        elif 24 < age < 40:
            res = 11 if gender == 'M' else 12
        elif 41 < age < 60:
            res = 13 if gender == 'M' else 14
        else:
            res = 15 if gender == 'M' else 16
    elif occupation in group3:
        if age < 23:
            res = 17 if gender == 'M' else 18
        elif 24 < age < 40:
            res = 19 if gender == 'M' else 20
        elif 41 < age < 60:
            res = 21 if gender == 'M' else 22
        else:
            res = 23 if gender == 'M' else 24
    elif occupation in group4:
        if age < 23:
            res = 25 if gender == 'M' else 26
        elif 24 < age < 40:
            res = 27 if gender == 'M' else 28
        elif 41 < age < 60:
            res = 29 if gender == 'M' else 30
        else:
            res = 31 if gender == 'M' else 32
    elif occupation in group5:
        if age < 23:
            res = 33 if gender == 'M' else 34
        elif 24 < age < 40:
            res = 35 if gender == 'M' else 36
        elif 41 < age < 60:
            res = 37 if gender == 'M' else 38
        else:
            res = 39 if gender == 'M' else 40

    return res - 1 if res > 9 else res

def movie_votes_demographic(grupos, ratings, movie_id, grupo):
    usuarios = [k for k, v in grupos.items() if v == grupo]
    return len(ratings[(ratings.user_id.isin(usuarios)) & (ratings.movie_id == movie_id)].index), ratings[(ratings.user_id.isin(usuarios)) & (ratings.movie_id == movie_id)].mean()['rating'], ratings[ratings.movie_id == movie_id].mean()['rating']

def genre_seen(films_df, ratings, user_id, genre_name):
    scores = []
    for film_id in ratings[ratings['user_id'] == user_id].movie_id.tolist():
        if films_df[films_df['movie_id'] == film_id][genre_name].tolist()[0] == 1:
            scores.append(ratings[(ratings['movie_id'] == film_id) & (ratings['user_id'] == user_id)]['rating'].tolist()[0])

    return sum(scores)/len(scores) if scores != [] else 0

def take(n, iterable):
    res = {}
    i = 0
    for k, v in iterable.items():
        if i >= n:
            break
        res[k] = v
        i += 1
        
    return res

def get_genres_score(users, generos, films_df, ratings):
    best_genres = {}
    for u in users:
        for i in generos['genre_name'].tolist():
            puntos = genre_seen(films_df, ratings, u, i)
            if puntos > 2.5:
                if i not in best_genres.keys():
                    best_genres[i] = puntos/len(users)
                else:
                    best_genres[i] += puntos/len(users)

    best_genres = {k: v for k, v in sorted(best_genres.items(), key=lambda item: item[1])}
    best_genres = take(6, best_genres)

    pref = []

    for genero in generos['genre_name'].tolist():
        add = False
        for k, v in best_genres.items():
            if genero == k:
                add = True
                break
        if add:
            pref.append(1)
        else:
            pref.append(0)

    return pref
    

def obtener_vecinos(preferencias, user, k=1):
    vecinos = [0]*k
    vecinos_score = [0]*k
    pref = preferencias[user]
    for i in range(0, preferencias.shape[0]):
        if i == user:
            continue
        pref_comp = np.matrix([pref, preferencias[i]])
        score = sum([1 if pref_comp[0,j] == pref_comp[1,j] else 0 for j in range(0, pref_comp.shape[1])])
        if score > min(vecinos_score):
            vecinos[vecinos_score.index(min(vecinos_score))] = i + 1
            vecinos_score[vecinos_score.index(min(vecinos_score))] = score
            
            
    return vecinos, vecinos_score

def get_dg_pref(dg, generos, films_df, ratings):
    res = []
    for x in set(dg.values()):
        u_dg = [k for k, v in dg.items() if v == x]
        scores = get_genres_score(u_dg, generos, films_df, ratings)
        res.append(scores)

    return np.matrix(res)
        



In [5]:
grupos_demograficos = {}
for user_id in users_df.user_id.unique().tolist():
    user = users_df[users_df.user_id == user_id]
    user_demo_group = get_user_type(user.gender.tolist()[0], user.age.tolist()[0], user.occupation.tolist()[0])
    grupos_demograficos[user_id] = user_demo_group

In [6]:
ratings = pd.read_csv(
    "../data/u1_base.txt", names=["user_id", "movie_id", "rating"], sep="\t"
)
generos = pd.read_csv("../data/genre.txt", names=["genre_id", "genre_name"], sep="\t")
all_genre = generos.genre_name.values.tolist()
all_genre = ["movie_id"] + all_genre + ["title"]
films_df = pd.read_csv("../data/items.txt",encoding="iso-8859-1" ,names=all_genre, sep="\t")

In [94]:
preferencias = []
for u in ratings.user_id.unique().tolist():
    best_genres = []
    for i in generos['genre_name'].tolist():
        puntos = genre_seen(films_df, ratings, u, i)
        if puntos > 2.5:
            best_genres.append((i, puntos))

    best_genres.sort(key=lambda a: a[1], reverse=True)
    best_genres = best_genres[:6]
    pref = []
    
    for genero in generos['genre_name'].tolist():
        add = False
        for bg in best_genres:
            if genero == bg[0]:
                add = True
                break
        if add:
            pref.append(1)
        else:
            pref.append(0)

    preferencias.append(pref)

In [97]:
preferencias = np.matrix(preferencias)

In [98]:
np.savez_compressed('preferencias', a=preferencias)

In [7]:
pref_loaded = np.load('preferencias.npz')

In [37]:
pref_dg = get_dg_pref(grupos_demograficos, generos, films_df, ratings)

In [61]:
pref_dg = np.load('preferencias_demografico.npz')

In [76]:
pref_hyb = []
for u in ratings.user_id.unique().tolist():
    pref = np.round(np.sum(np.matrix([pref_loaded['a'][u-1], pref_dg['a'][grupos_demograficos[u]-1]]), axis=0)/2, 0).tolist()[0]
    pref_hyb.append(pref)

pref_hyb = np.matrix(pref_hyb)

In [85]:
pref_hyb = np.load('preferencias_hibrido.npz')

In [ ]:
puntuaciones={}
for user_id in users_df.user_id.unique().tolist():
    puntuaciones[user_id] = {}
    print(user_id)
    dem_group = grupos_demograficos[user_id]
    for movie_id in films_df.movie_id.unique().tolist():
        v, r, c = movie_votes_demographic(grupos_demograficos, ratings, movie_id, dem_group)
        puntuaciones[user_id][movie_id] = v/((v+5)*r) + ((5/(5+v)) * c)

In [108]:
obtener_vecinos(pref_loaded['a'], 3, 3)

([58, 47, 3], [17, 17, 17])

In [96]:
pref_dg['a']

array([[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0],
       [0,

In [1]:
from recomendador import Recomendador
import numpy as np

In [2]:
recom = Recomendador(data_path='../data/')
recom.load_grupos_demograficos()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [3]:
recom.load_preferencias(path='../data/preferencias.npz', path_dg='../data/preferencias_demografico.npz', path_hyb='../data/preferencias_hibrido.npz')


In [9]:
recom.save_user_pref(2, [0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

[0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
recom.obtener_vecinos(recom.preferencias_coop, 2, 1)

([3], [1.0])

In [11]:
recom.preferencias_coop[1]

array([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [34]:
np.append(recom.preferencias_coop, [[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]], axis=0)[943]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [12]:
recom.get_user_films(1)

[1,
 2,
 3,
 4,
 5,
 7,
 8,
 9,
 11,
 13,
 15,
 16,
 18,
 19,
 21,
 22,
 25,
 26,
 28,
 29,
 30,
 32,
 34,
 35,
 37,
 38,
 40,
 41,
 42,
 43,
 45,
 46,
 48,
 50,
 52,
 55,
 57,
 58,
 59,
 63,
 66,
 68,
 71,
 75,
 77,
 79,
 83,
 87,
 88,
 89,
 93,
 94,
 95,
 99,
 101,
 105,
 106,
 109,
 110,
 111,
 115,
 116,
 119,
 122,
 123,
 124,
 126,
 127,
 131,
 133,
 135,
 136,
 137,
 138,
 139,
 141,
 142,
 144,
 146,
 147,
 149,
 152,
 153,
 156,
 158,
 162,
 165,
 166,
 167,
 168,
 169,
 172,
 173,
 176,
 178,
 179,
 181,
 182,
 187,
 191,
 192,
 194,
 195,
 197,
 198,
 199,
 203,
 204,
 205,
 207,
 211,
 216,
 217,
 220,
 223,
 231,
 234,
 237,
 238,
 239,
 240,
 244,
 245,
 246,
 247,
 249,
 251,
 256,
 257,
 261,
 263,
 268,
 269,
 270,
 271,
 274]

In [13]:
recom.get_user_film_rate(1,2)

3